In [1]:

####################################################################################################
####################################################################################################

import sys
import os

import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint

from IPython.display import clear_output
from IPython.display import display

sys.path.append(os.path.abspath("../"))
from pax_utils import event_utils
from pax_utils import file_utils
from pax_utils import interaction_utils
from pax_utils import s1s2_utils
from pax_utils import numeric_utils
from pax_utils import waveform_pax_utils
from pax_utils import waveform_utils

sys.path.append(os.path.abspath("../pax"))
from pax import core



In [ ]:

####################################################################################################
####################################################################################################

f = plt.figure(figsize=(20, 10))

def processPklEvents(zipfilename, iZip, dir_waveforms_s2):

    ################################################################################################
    ################################################################################################
    
    lst_pkl_files  = file_utils.getListOfPklFilesFromZipFile(zipfilename, nEventsPerFile)

    
    ################################################################################################
    ################################################################################################
    
    sum_wf = None
    #event  = None
    df_zip_merged   = pd.DataFrame()
    df_s2_waveforms = pd.DataFrame()
        
    ################################################################################################
    ################################################################################################
    
    for iPklFile in range(0, len(lst_pkl_files)):
    
      
        ############################################################################################
        ############################################################################################

        event_number = iZip*1000 + iPklFile
        
        file_out_s2_waveforms = dir_waveforms_s2 + '/' + 'event' + format(event_number, '04d') + '_S2waveforms' + '.pkl'
        
        
        ############################################################################################
        ############################################################################################
        
        pklfilename  = lst_pkl_files[iPklFile]
        event        = file_utils.getPaxEventFromPklFileInZipArchive(zipfilename, pklfilename)
        jsonfilename = os.path.dirname(zipfilename) + '/pax_info.json'
        cfg          = event_utils.getConfig(jsonfilename)

        
        ############################################################################################
        ############################################################################################
        
        print(" -> Event Number: " + str(iPklFile))
        clear_output(wait=True)
        
        
        ############################################################################################
        ############################################################################################
        
        interactions  = event.interactions
        nInteractions = len(interactions)
        
        #if (nInteractions != 1): continue
            
            
        ############################################################################################
        ############################################################################################
        
        df_pkl_event  = event_utils.getEventDataFrameFromEvent(event)
        df_pkl_intr   = interaction_utils.getInteractionDataFrameFromEvent(event)
        df_pkl_s2s    = s1s2_utils.getS2integralsDataFrame(event, 127)
        
                  
        ############################################################################################
        ############################################################################################
    
        df_pkl_merged = df_pkl_event.merge(df_pkl_intr).merge(df_pkl_s2s)
        df_zip_merged = df_zip_merged.append(df_pkl_merged)
        
        
        
        ############################################################################################
        ############################################################################################
        
        df_channels_waveforms_top = pd.DataFrame()
        
        
        ############################################################################################
        ############################################################################################
        
        if (nInteractions < 1):
            
            df_channels_waveforms_top.to_pickle(file_out_s2_waveforms)
            
            continue
        
        
        ############################################################################################
        ############################################################################################
        
        left  = event.main_s2.left
        right = event.main_s2.right

        
        ############################################################################################
        # Get summed S2 waveform from event, PAX
        ############################################################################################
        
        arr_summed_waveform_top_evt = waveform_pax_utils.getSummedWaveformFromEvent(event, 'tpc_top')
        arr_summed_waveform_top_evt = arr_summed_waveform_top_evt[left:right]
        
        
        ############################################################################################
        # Get summed S2 waveform from PAX
        ############################################################################################
        
        #arr_summed_waveform_top_pax = waveform_pax_utils.SumWaveformPAX(event)
        #arr_summed_waveform_top_pax = arr_summed_waveform_top_evt[left:right]
                
            
        ############################################################################################
        # Get dataframe of S2 waveform for each PMT channel
        ############################################################################################
        
        df_channels_waveforms_top     = waveform_utils.getChannelsWaveformsDataFrame(event, cfg, 'top', False)
        df_channels_waveforms_top_all = waveform_utils.addEmptyChannelsToDataFrame(df_channels_waveforms_top)
        
        
        ############################################################################################
        # Get summed S2 waveform from dataframe
        ############################################################################################
        
        arr_summed_waveform_top_df  = waveform_utils.getSummedWaveformFromDataFrame(df_channels_waveforms_top_all, event)
        arr_summed_waveform_top_df  = arr_summed_waveform_top_df[left:right]
        
        
        ############################################################################################
        # Check that the S2 summed waveform from the event and dataframe are equal
        ############################################################################################
        
        wf_arrs_equal = numeric_utils.compareArrays(arr_summed_waveform_top_evt, arr_summed_waveform_top_df)
        
        
        ############################################################################################
        # Check that the integrals of the S2 summed waveform from the event and dataframe are equal
        ############################################################################################
        
        #sum_summed_waveform_top_pax    = np.sum(arr_summed_waveform_top_pax)
        sum_summed_waveform_top_evt     = np.sum(arr_summed_waveform_top_evt)
        sum_summed_waveform_top_df      = np.sum(arr_summed_waveform_top_df)
        
        #
        #sum_summed_waveforms_equal = numeric_utils.compareFloats(sum_summed_waveform_top_evt, sum_summed_waveform_top_df, 1e-3)
        sum_summed_waveforms_equal = abs(sum_summed_waveform_top_evt - sum_summed_waveform_top_df) / sum_summed_waveform_top_evt < 0.1
            
        
        ############################################################################################
        # Check that the per-channel S2 integrals from the event and dataframe are equal
        ############################################################################################

        arr_s2integrals_evt = df_pkl_s2s.iloc[0][1:].as_matrix().astype(np.float32)
        arr_s2integrals_df  = df_channels_waveforms_top_all[:]['sum'].as_matrix().astype(np.float32)
        
        arr_s2integrals_equal = numeric_utils.compareArrays(arr_s2integrals_evt, arr_s2integrals_df)
        
        
        ############################################################################################
        # Check that the sum of the per-channel S2 integrals from the event and dataframe are equal
        ############################################################################################
        
        sum_s2integrals_evt = np.sum(arr_s2integrals_evt)
        sum_s2integrals_df  = np.sum(arr_s2integrals_df)
        
        # this fails use weaker condition for now
        #sum_s2integrals_equal = numeric_utils.compareFloats(sum_s2integrals_evt, sum_s2integrals_df)
        sum_s2integrals_equal = abs(sum_s2integrals_evt - sum_s2integrals_df) / sum_s2integrals_df < 0.1
        
        
        ############################################################################################
        ############################################################################################
        
        verbose = False
        sane    = wf_arrs_equal and sum_summed_waveforms_equal and arr_s2integrals_equal and sum_s2integrals_equal
        
        if (verbose or not sane):
            
            print()
            print("Event:                                   " + str(event_number))
            print("Integral of Summed Waveform Event:       " + str(sum_summed_waveform_top_evt))
            print("Integral of Summed Waveform DF:          " + str(sum_summed_waveform_top_df))
            print("Sum of S2 Integrals over channels Event: " + str(sum_s2integrals_evt))
            print("Sum of S2 Integrals over channels DF:    " + str(sum_s2integrals_df))
            print()
        
        
        ############################################################################################
        ############################################################################################
        
        assert(wf_arrs_equal)
        assert(sum_summed_waveforms_equal)
        assert(arr_s2integrals_equal)
        assert(sum_s2integrals_equal) # this fails
        
        
        ############################################################################################
        # Save S2 Waveforms
        ############################################################################################

        df_channels_waveforms_top.to_pickle(file_out_s2_waveforms)
        
        #display(df_channels_waveforms_top[0:5][:])

        
        ############################################################################################
        # End loop on PKL files in ZIP File
        ############################################################################################
        
        continue
      
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged.reset_index(inplace=True, drop=True)
       
    return df_zip_merged, df_s2_waveforms

    

In [ ]:
import glob

####################################################################################################
# input
####################################################################################################

nEventsPerFile = 1000
nFilesZip      = 2 #200
if (nFilesZip == -1): nFilesZip = len(lst_contents)
nEvents        = nEventsPerFile * nFilesZip

dir_input    = '../pax_run/pax_output/Mar25/'
dir_format   = dir_input + "instructions_" + ('[0-9]' * 6)
file_format  = 'XENON1T-0-000000000-000000999-000001000.zip'
lst_contents = glob.glob(dir_format)


####################################################################################################
# output
####################################################################################################

ver              = datetime.datetime.now().strftime("%y%m%d%H%M")
ver              = 'test'
dir_out_pkl      = 'merged' + '/'
file_pkl         = dir_out_pkl + 'merged_pax_' + str(nEvents % 1000) + 'k_' + ver + '.pkl'
dir_waveforms    = dir_out_pkl + '/' + 'waveforms_' + ver
dir_waveforms_s2 = dir_waveforms + '/' + 's2'

if(not os.path.isdir(dir_waveforms)):
    os.mkdir(dir_waveforms)
    if(not os.path.isdir(dir_waveforms_s2)):
        os.mkdir(dir_waveforms_s2)
    else:
        print("\nDirectory '" + dir_waveforms_s2 + "' exists!\n") 
else:
    print("\nDirectory '" + dir_waveforms + "' exists!\n")


#print()
#print("PAX input files: " + str(nFilesZip))
#print()


####################################################################################################
####################################################################################################

df = pd.DataFrame()

for iZip in range(0, nFilesZip):
    
    ################################################################################################
    ################################################################################################
    
    zipfilename = lst_contents[iZip] + '/' + file_format
    
    if (not os.path.exists(zipfilename)):
        
        print("Error! File: '" + str(zipfilename) + "' does not exist.")
    
        continue
        
    print("Zip File: '" + str(zipfilename) + "'")
    
    
    ################################################################################################
    ################################################################################################
    
    df_zip_merged, df_s2_waveforms = processPklEvents(zipfilename, iZip, dir_waveforms_s2)

    
    ################################################################################################
    ################################################################################################
    
    df = df.append(df_zip_merged)
    
    continue
    

print("Done")


####################################################################################################
# Write
####################################################################################################

#display(df[0:1][:])
print(file_pkl)

df.to_pickle(file_pkl)



 -> Event Number: 175
